In [13]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [14]:
df_raw = pd.read_csv("../dataset/electricity.csv")
df_raw.head(5)

,date,0,1,2,3,4,5,6,7,8,...,311,312,313,314,315,316,317,318,319,OT
0,2016-07-01 02:00:00,14.0,69.0,234.0,415.0,215.0,1056.0,29.0,840.0,226.0,...,676.0,372.0,80100.0,4719.0,5002.0,48.0,38.0,1558.0,182.0,2162.0
1,2016-07-01 03:00:00,18.0,92.0,312.0,556.0,292.0,1363.0,29.0,1102.0,271.0,...,805.0,452.0,95200.0,4643.0,6617.0,65.0,47.0,2177.0,253.0,2835.0
2,2016-07-01 04:00:00,21.0,96.0,312.0,560.0,272.0,1240.0,29.0,1025.0,270.0,...,817.0,430.0,96600.0,4285.0,6571.0,64.0,43.0,2193.0,218.0,2764.0
3,2016-07-01 05:00:00,20.0,92.0,312.0,443.0,213.0,845.0,24.0,833.0,179.0,...,801.0,291.0,94500.0,4222.0,6365.0,65.0,39.0,1315.0,195.0,2735.0
4,2016-07-01 06:00:00,22.0,91.0,312.0,346.0,190.0,647.0,16.0,733.0,186.0,...,807.0,279.0,91300.0,4116.0,6298.0,75.0,40.0,1378.0,191.0,2721.0


In [15]:
class Dataset_Custom(Dataset):
    def __init__(
        self,
        root_path="",
        flag="train",
        size=None,
        features="M",
        data_path="ETTh1.csv",
        target="OT",
        scale=True,
        timeenc=0,
        freq="h",
        train_only=False,
    ):
        # size [seq_len, label_len, pred_len]
        # info
        if size is None:
            self.seq_len = 336
            self.label_len = 48
            self.pred_len = 96
        else:
            self.seq_len = size[0]
            self.label_len = size[1]  # 计算损失用的
            self.pred_len = size[2]
        # init
        assert flag in ["train", "test", "val"]
        type_map = {"train": 0, "val": 1, "test": 2}
        self.set_type = type_map[flag]

        self.features = features
        self.target = target
        self.scale = scale
        self.timeenc = timeenc
        self.freq = freq
        self.train_only = train_only

        self.root_path = root_path
        self.data_path = data_path


self = Dataset_Custom()

In [16]:
cols = list(df_raw.columns)
cols.remove("OT")
cols.remove("date")
print(cols)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '15

In [17]:
num_train = int(len(df_raw) * (0.7 if not self.train_only else 1))
num_test = int(len(df_raw) * 0.2)
num_vali = len(df_raw) - num_train - num_test
border1s = [0, num_train - self.seq_len, len(df_raw) - num_test - self.seq_len]
border2s = [num_train, num_train + num_vali, len(df_raw)]
border1 = border1s[self.set_type]
border2 = border2s[self.set_type]

In [18]:
if self.features == 'M' or self.features == 'MS':
    df_raw = df_raw[['date'] + cols]
    cols_data = df_raw.columns[1:]
    df_data = df_raw[cols_data]
elif self.features == 'S':
    df_raw = df_raw[['date'] + cols + [self.target]]
    df_data = df_raw[[self.target]]

In [19]:
data = df_data.values
df_stamp = df_raw[['date']][border1:border2]
df_stamp['date'] = pd.to_datetime(df_stamp.date)
if self.timeenc == 0:
    df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
    df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
    df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
    df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
    data_stamp = df_stamp.drop(['date'], 1).values
elif self.timeenc == 1:
    data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=self.freq)
    data_stamp = data_stamp.transpose(1, 0)

self.data_x = data[border1:border2]
self.data_y = data[border1:border2]
self.data_stamp = data_stamp

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2344\2929428812.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_stamp = df_stamp.drop(['date'], 1).values


In [20]:
self.data_x.shape

(18412, 320)

In [21]:
self.data_y.shape

(18412, 320)

In [22]:
self.data_stamp.shape

(18412, 4)

In [27]:
def getitem(index):
    s_begin = index
    s_end = s_begin + self.seq_len
    r_begin = s_end - self.label_len
    r_end = r_begin + self.label_len + self.pred_len

    print(f"<{s_begin=},{s_end=}>|<{r_begin=},{r_end=}>")

    seq_x = self.data_x[s_begin:s_end]
    seq_y = self.data_y[r_begin:r_end]
    seq_x_mark = self.data_stamp[s_begin:s_end]
    seq_y_mark = self.data_stamp[r_begin:r_end]

    print(f"<{seq_x.shape=},{seq_y.shape=}><{seq_x_mark.shape=}{seq_y_mark.shape=}>")

In [28]:
getitem(0)

<s_begin=0,s_end=336>|<r_begin=288,r_end=432>
<seq_x.shape=(336, 320),seq_y.shape=(144, 320)><seq_x_mark.shape=(336, 4)seq_y_mark.shape=(144, 4)>


In [29]:
getitem(1)

<s_begin=1,s_end=337>|<r_begin=289,r_end=433>
<seq_x.shape=(336, 320),seq_y.shape=(144, 320)><seq_x_mark.shape=(336, 4)seq_y_mark.shape=(144, 4)>


In [30]:
getitem(2)

<s_begin=2,s_end=338>|<r_begin=290,r_end=434>
<seq_x.shape=(336, 320),seq_y.shape=(144, 320)><seq_x_mark.shape=(336, 4)seq_y_mark.shape=(144, 4)>
